# H02: Power Correlation (PowCorr)

**Duration**: 45 minutes  
**Prerequisites**: H01 (Envelope Correlation), A03 (Power Spectrum)  
**Next**: Workshop completion / Advanced topics

---

## Learning Objectives

By the end of this notebook, you will be able to:

1. Distinguish power correlation from envelope correlation
2. Define power correlation as correlation of squared envelopes / band power
3. Implement power correlation computation
4. Understand time-windowed vs instantaneous approaches
5. Apply power correlation to hyperscanning analysis
6. Choose between envelope correlation and power correlation
7. Complete the amplitude connectivity toolkit

In [ ]:
# Required imports
import numpy as np
from numpy.typing import NDArray
import matplotlib.pyplot as plt
from scipy import signal
from scipy import stats
from typing import Any, Tuple

# Visualization settings
plt.style.use('seaborn-v0_8-whitegrid')

# Color palette
COLORS = {
    'signal_1': '#2E86AB',
    'signal_2': '#E94F37',
    'accent': '#A23B72',
    'highlight': '#F18F01',
    'warning': '#C73E1D',
    'subject_1': '#2E86AB',
    'subject_2': '#E94F37',
}

# Random seed for reproducibility
np.random.seed(42)

---

## Section 1: Introduction — Power vs Amplitude

In H01, we learned about **envelope correlation**, which measures the correlation between amplitude envelopes A(t). Now we explore a closely related metric: **power correlation**.

The key distinction:
- **Amplitude**: A(t) = |z(t)| where z is the analytic signal
- **Power**: P(t) = A(t)² = |z(t)|²

**Why power?** There are several reasons to consider power instead of amplitude:

1. **Power is standard in spectral analysis**: When we compute PSDs, we work with power, not amplitude
2. **Energy relationship**: Power is more directly related to signal "energy"
3. **Emphasis on peaks**: Squaring emphasizes larger amplitude fluctuations more than smaller ones

**Power correlation** is simply the Pearson correlation between power time series. If amplitudes are correlated, powers will be too (squaring is monotonic), but the values won't be identical—squaring changes the distribution and emphasizes extremes.

In [ ]:
# Helper functions from H01

def bandpass_filter(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    order: int = 4
) -> NDArray[np.float64]:
    """Apply bandpass filter to signal."""
    nyq = fs / 2
    low, high = band[0] / nyq, band[1] / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data, axis=-1)


def extract_envelope(
    data: NDArray[np.float64]
) -> NDArray[np.float64]:
    """Extract amplitude envelope using Hilbert transform."""
    analytic = signal.hilbert(data, axis=-1)
    return np.abs(analytic)


def extract_phase(
    data: NDArray[np.float64]
) -> NDArray[np.float64]:
    """Extract instantaneous phase using Hilbert transform."""
    analytic = signal.hilbert(data, axis=-1)
    return np.angle(analytic)

In [ ]:
# Visualization 1: Signal → Amplitude → Power

np.random.seed(42)
fs = 500.0
duration = 4.0
t = np.arange(0, duration, 1/fs)

# Create amplitude-modulated signal
modulation = 1 + 0.6 * np.sin(2 * np.pi * 0.3 * t)
carrier = np.sin(2 * np.pi * 10 * t)
sig = modulation * carrier + 0.15 * np.random.randn(len(t))

# Filter and extract envelope/power
sig_filt = bandpass_filter(sig, fs, (8, 12))
envelope = extract_envelope(sig_filt)
power = envelope ** 2

fig, axes = plt.subplots(3, 1, figsize=(12, 9), sharex=True)

# Panel 1: Filtered signal
ax1 = axes[0]
ax1.plot(t, sig_filt, color=COLORS['signal_1'], linewidth=1, alpha=0.8)
ax1.set_ylabel('Signal x(t)', fontsize=11)
ax1.set_title('Filtered Signal (8-12 Hz)', fontsize=12, fontweight='bold')

# Panel 2: Amplitude envelope
ax2 = axes[1]
ax2.plot(t, sig_filt, color=COLORS['signal_1'], linewidth=0.5, alpha=0.3)
ax2.plot(t, envelope, color=COLORS['highlight'], linewidth=2.5, label='Amplitude A(t)')
ax2.plot(t, -envelope, color=COLORS['highlight'], linewidth=2.5)
ax2.set_ylabel('Amplitude A(t)', fontsize=11)
ax2.set_title('Amplitude Envelope', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right')

# Panel 3: Power
ax3 = axes[2]
ax3.fill_between(t, 0, power, color=COLORS['accent'], alpha=0.5)
ax3.plot(t, power, color=COLORS['accent'], linewidth=2, label='Power P(t) = A(t)²')
ax3.set_xlabel('Time (s)', fontsize=11)
ax3.set_ylabel('Power A(t)²', fontsize=11)
ax3.set_title('Instantaneous Power (amplitude squared)', fontsize=12, fontweight='bold')
ax3.legend(loc='upper right')

# Annotate peak emphasis
peak_idx = np.argmax(envelope[500:1500]) + 500
ax2.annotate('Peak amplitude', xy=(t[peak_idx], envelope[peak_idx]), 
             xytext=(t[peak_idx] + 0.5, envelope[peak_idx] + 0.3),
             arrowprops=dict(arrowstyle='->', color='black'), fontsize=10)
ax3.annotate('Peak MORE emphasized\nin power (squared)', xy=(t[peak_idx], power[peak_idx]), 
             xytext=(t[peak_idx] + 0.5, power[peak_idx] + 0.5),
             arrowprops=dict(arrowstyle='->', color='black'), fontsize=10)

plt.suptitle('Signal → Amplitude → Power', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("💡 Power (A²) emphasizes large fluctuations more than amplitude (A)")
print(f"   Max amplitude: {envelope.max():.2f}")
print(f"   Max power: {power.max():.2f} = {envelope.max():.2f}² = {envelope.max()**2:.2f}")

---

## Section 2: Power Correlation Definition

**Power correlation** is defined as:

$$PowCorr = Pearson(P_x, P_y) = Pearson(A_x^2, A_y^2)$$

where $P(t) = A(t)^2 = |z(t)|^2$ is the instantaneous power.

### Properties:

- **Range**: -1 to +1 (like envelope correlation)
- **PowCorr = +1**: Powers perfectly positively correlated
- **PowCorr = 0**: No linear power relationship
- **PowCorr < 0**: Anti-correlated powers (rare)

### Relationship to envelope correlation:

- Usually highly correlated with CCorr
- PowCorr is typically slightly higher (squaring reduces noise relative to signal)
- Not identical—they have different emphases

In [ ]:
# Visualization 2: Scatter of CCorr vs PowCorr for many signal pairs

np.random.seed(42)
n_pairs = 100
n_samples = 5000
fs = 500.0
band = (8, 12)
t = np.arange(n_samples) / fs

ccorr_values = []
powcorr_values = []

for _ in range(n_pairs):
    # Random coupling strength
    coupling = np.random.uniform(0, 1)
    
    # Generate signals with varying amplitude coupling
    shared_mod = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
    indep_mod_x = 1 + 0.5 * np.sin(2 * np.pi * (0.3 + np.random.uniform(-0.1, 0.1)) * t + np.random.uniform(0, 2*np.pi))
    indep_mod_y = 1 + 0.5 * np.sin(2 * np.pi * (0.3 + np.random.uniform(-0.1, 0.1)) * t + np.random.uniform(0, 2*np.pi))
    
    env_x = np.sqrt(coupling) * shared_mod + np.sqrt(1-coupling) * indep_mod_x
    env_y = np.sqrt(coupling) * shared_mod + np.sqrt(1-coupling) * indep_mod_y
    
    x = env_x * np.sin(2 * np.pi * 10 * t + np.random.uniform(0, 2*np.pi)) + 0.2 * np.random.randn(n_samples)
    y = env_y * np.sin(2 * np.pi * 10 * t + np.random.uniform(0, 2*np.pi)) + 0.2 * np.random.randn(n_samples)
    
    # Filter and extract
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    env_x_meas = extract_envelope(x_filt)
    env_y_meas = extract_envelope(y_filt)
    pow_x = env_x_meas ** 2
    pow_y = env_y_meas ** 2
    
    # Compute correlations
    ccorr = stats.pearsonr(env_x_meas, env_y_meas)[0]
    powcorr = stats.pearsonr(pow_x, pow_y)[0]
    
    ccorr_values.append(ccorr)
    powcorr_values.append(powcorr)

fig, ax = plt.subplots(figsize=(8, 8))

ax.scatter(ccorr_values, powcorr_values, s=50, alpha=0.6, color=COLORS['signal_1'], edgecolors='white')
ax.plot([-0.2, 1], [-0.2, 1], 'k--', linewidth=2, label='y = x (identity)')

# Fit line
z = np.polyfit(ccorr_values, powcorr_values, 1)
p = np.poly1d(z)
x_line = np.linspace(-0.2, 1, 100)
ax.plot(x_line, p(x_line), color=COLORS['highlight'], linewidth=2, 
        label=f'Fit: y = {z[0]:.2f}x + {z[1]:.2f}')

ax.set_xlabel('Envelope Correlation (CCorr)', fontsize=12)
ax.set_ylabel('Power Correlation (PowCorr)', fontsize=12)
ax.set_title('Envelope vs Power Correlation: Related but Distinct', fontsize=14, fontweight='bold')
ax.legend(loc='lower right', fontsize=10)
ax.set_xlim(-0.2, 1)
ax.set_ylim(-0.2, 1)
ax.set_aspect('equal')

# Correlation
r = np.corrcoef(ccorr_values, powcorr_values)[0, 1]
ax.text(0.05, 0.95, f'r = {r:.3f}', transform=ax.transAxes, fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"Correlation between CCorr and PowCorr: r = {r:.3f}")
print(f"Mean difference (PowCorr - CCorr): {np.mean(np.array(powcorr_values) - np.array(ccorr_values)):.4f}")

---

## Section 3: Two Approaches to Power Correlation

There are two main approaches to computing power correlation:

### Approach 1: Instantaneous Power Correlation
- Compute P(t) = A(t)² at each time point
- Correlate power time series
- **Advantage**: High temporal resolution

### Approach 2: Windowed Band Power Correlation
- Divide signal into epochs/windows
- Compute band power for each window
- Correlate power values across windows
- **Advantage**: More robust estimates, lower variance

**When to use which?**
- **Instantaneous**: Dynamic connectivity studies, short recordings
- **Windowed**: More stable estimates, trial-based designs, longer recordings

In [ ]:
# Visualization 3: Instantaneous vs windowed power

np.random.seed(42)
fs = 500.0
duration = 10.0
t = np.arange(0, duration, 1/fs)
n_samples = len(t)

# Create signal with power modulation
modulation = 1 + 0.6 * np.sin(2 * np.pi * 0.2 * t)
sig = modulation * np.sin(2 * np.pi * 10 * t) + 0.2 * np.random.randn(n_samples)

# Filter and extract
sig_filt = bandpass_filter(sig, fs, (8, 12))
envelope = extract_envelope(sig_filt)
inst_power = envelope ** 2

# Windowed power
window_sec = 1.0
window_samples = int(window_sec * fs)
step = window_samples // 2  # 50% overlap

window_centers = []
windowed_power = []

for start in range(0, n_samples - window_samples + 1, step):
    end = start + window_samples
    window_centers.append((start + end) / 2 / fs)
    windowed_power.append(np.mean(inst_power[start:end]))

window_centers = np.array(window_centers)
windowed_power = np.array(windowed_power)

fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Instantaneous power
ax1 = axes[0]
ax1.plot(t, inst_power, color=COLORS['signal_1'], linewidth=1, alpha=0.8)
ax1.fill_between(t, 0, inst_power, color=COLORS['signal_1'], alpha=0.3)
ax1.set_ylabel('Power', fontsize=11)
ax1.set_title('Instantaneous Power P(t) = A(t)²\n(High temporal resolution, more variable)', 
              fontsize=12, fontweight='bold')

# Windowed power
ax2 = axes[1]
ax2.plot(t, inst_power, color=COLORS['signal_1'], linewidth=0.5, alpha=0.3, label='Instantaneous')
ax2.step(window_centers, windowed_power, where='mid', color=COLORS['signal_2'], 
         linewidth=2.5, label=f'Windowed ({window_sec}s windows)')
ax2.scatter(window_centers, windowed_power, color=COLORS['signal_2'], s=50, zorder=5)
ax2.set_xlabel('Time (s)', fontsize=11)
ax2.set_ylabel('Power', fontsize=11)
ax2.set_title('Windowed Band Power\n(Lower resolution, more robust)', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right')

plt.suptitle('Instantaneous vs Windowed Power', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---

## Section 4: Implementing Power Correlation

Let's implement both approaches.

In [ ]:
def compute_instantaneous_power(
    sig: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> NDArray[np.float64]:
    """
    Compute instantaneous power time series.
    
    Parameters
    ----------
    sig : NDArray[np.float64]
        Input signal
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
        
    Returns
    -------
    NDArray[np.float64]
        Instantaneous power P(t) = A(t)²
    """
    sig_filt = bandpass_filter(sig, fs, band)
    envelope = extract_envelope(sig_filt)
    return envelope ** 2


def compute_windowed_band_power(
    sig: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    window_sec: float = 1.0,
    overlap: float = 0.5
) -> Tuple[NDArray[np.float64], NDArray[np.float64]]:
    """
    Compute band power in sliding windows.
    
    Parameters
    ----------
    sig : NDArray[np.float64]
        Input signal
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    window_sec : float, optional
        Window length in seconds (default: 1.0)
    overlap : float, optional
        Overlap fraction 0-1 (default: 0.5)
        
    Returns
    -------
    Tuple[NDArray[np.float64], NDArray[np.float64]]
        (time_centers, power_values)
    """
    # First compute instantaneous power
    inst_power = compute_instantaneous_power(sig, fs, band)
    
    window_samples = int(window_sec * fs)
    step_samples = int(window_samples * (1 - overlap))
    n_samples = len(sig)
    
    time_centers = []
    power_values = []
    
    for start in range(0, n_samples - window_samples + 1, step_samples):
        end = start + window_samples
        time_centers.append((start + end) / 2 / fs)
        power_values.append(np.mean(inst_power[start:end]))
    
    return np.array(time_centers), np.array(power_values)


def compute_power_correlation(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    method: str = "instantaneous",
    window_sec: float = 1.0
) -> float:
    """
    Compute power correlation between two signals.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal
    y : NDArray[np.float64]
        Second signal
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    method : str, optional
        'instantaneous' or 'windowed' (default: 'instantaneous')
    window_sec : float, optional
        Window length for windowed method (default: 1.0)
        
    Returns
    -------
    float
        Power correlation in range [-1, 1]
    """
    if method == "instantaneous":
        pow_x = compute_instantaneous_power(x, fs, band)
        pow_y = compute_instantaneous_power(y, fs, band)
    elif method == "windowed":
        _, pow_x = compute_windowed_band_power(x, fs, band, window_sec)
        _, pow_y = compute_windowed_band_power(y, fs, band, window_sec)
    else:
        raise ValueError(f"Unknown method: {method}. Use 'instantaneous' or 'windowed'.")
    
    # Handle constant signals
    if np.std(pow_x) == 0 or np.std(pow_y) == 0:
        return 0.0
    
    corr, _ = stats.pearsonr(pow_x, pow_y)
    return float(corr)

In [ ]:
# Visualization 4: Computation pipeline

fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 10)
ax.set_ylim(0, 10)
ax.axis('off')

# Helper function for boxes
def draw_box(ax, x, y, w, h, text, color, fontsize=9):
    rect = plt.Rectangle((x-w/2, y-h/2), w, h, fill=True, 
                          facecolor=color, edgecolor='black', linewidth=2, alpha=0.8)
    ax.add_patch(rect)
    ax.text(x, y, text, ha='center', va='center', fontsize=fontsize, fontweight='bold', wrap=True)

def draw_arrow(ax, x1, y1, x2, y2, text=''):
    ax.annotate('', xy=(x2, y2), xytext=(x1, y1),
                arrowprops=dict(arrowstyle='->', color='black', lw=2))
    if text:
        mid_x, mid_y = (x1+x2)/2, (y1+y2)/2
        ax.text(mid_x + 0.3, mid_y, text, fontsize=8)

# Shared start
draw_box(ax, 5, 9, 3, 0.8, 'Raw Signals\nx(t), y(t)', '#E8E8E8', 10)
draw_arrow(ax, 5, 8.6, 5, 8)
draw_box(ax, 5, 7.5, 3, 0.8, 'Bandpass Filter', COLORS['signal_1'])
draw_arrow(ax, 5, 7.1, 5, 6.5)
draw_box(ax, 5, 6, 3, 0.8, 'Hilbert Transform\n→ Analytic signal', COLORS['signal_1'])
draw_arrow(ax, 5, 5.6, 5, 5)
draw_box(ax, 5, 4.5, 3, 0.8, 'Amplitude Envelope\nA(t) = |z(t)|', COLORS['signal_1'])
draw_arrow(ax, 5, 4.1, 5, 3.5)
draw_box(ax, 5, 3, 3, 0.8, 'Square\nP(t) = A(t)²', COLORS['highlight'])

# Branch
draw_arrow(ax, 5, 2.6, 3, 2)
draw_arrow(ax, 5, 2.6, 7, 2)

# Instantaneous path
draw_box(ax, 3, 1.5, 2.5, 0.8, 'Instantaneous\nP(t)', COLORS['signal_1'])
draw_arrow(ax, 3, 1.1, 3, 0.5)
draw_box(ax, 3, 0, 2.5, 0.8, 'Pearson(Px, Py)', COLORS['accent'])

# Windowed path
draw_box(ax, 7, 1.5, 2.5, 0.8, 'Window Average\nmean(P)', COLORS['signal_2'])
draw_arrow(ax, 7, 1.1, 7, 0.5)
draw_box(ax, 7, 0, 2.5, 0.8, 'Pearson(Px, Py)', COLORS['accent'])

# Labels
ax.text(3, -0.8, 'INSTANTANEOUS', ha='center', fontsize=11, fontweight='bold', color=COLORS['signal_1'])
ax.text(7, -0.8, 'WINDOWED', ha='center', fontsize=11, fontweight='bold', color=COLORS['signal_2'])

ax.set_title('Power Correlation Computation Methods', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

---

## Section 5: Power Correlation with Examples

In [ ]:
def generate_power_correlated_signals(
    n_samples: int,
    fs: float,
    frequency: float,
    power_correlation_target: float,
    modulation_freq: float = 0.5,
    seed: int | None = None
) -> Tuple[NDArray[np.float64], NDArray[np.float64]]:
    """
    Generate two signals with specified power correlation.
    
    Parameters
    ----------
    n_samples : int
        Number of samples
    fs : float
        Sampling frequency in Hz
    frequency : float
        Carrier frequency in Hz
    power_correlation_target : float
        Desired power correlation (0 to 1)
    modulation_freq : float, optional
        Modulation frequency in Hz (default: 0.5)
    seed : int, optional
        Random seed
        
    Returns
    -------
    Tuple[NDArray[np.float64], NDArray[np.float64]]
        Two signals with correlated power
    """
    if seed is not None:
        np.random.seed(seed)
    
    t = np.arange(n_samples) / fs
    
    # Shared and independent modulation
    shared = 1 + 0.5 * np.sin(2 * np.pi * modulation_freq * t)
    indep_x = 1 + 0.5 * np.sin(2 * np.pi * modulation_freq * 1.3 * t + np.random.uniform(0, 2*np.pi))
    indep_y = 1 + 0.5 * np.sin(2 * np.pi * modulation_freq * 1.7 * t + np.random.uniform(0, 2*np.pi))
    
    # Mix based on target
    shared_weight = np.sqrt(max(0, power_correlation_target))
    indep_weight = np.sqrt(1 - max(0, power_correlation_target))
    
    env_x = shared_weight * shared + indep_weight * indep_x
    env_y = shared_weight * shared + indep_weight * indep_y
    
    # Ensure positive
    env_x = np.maximum(env_x, 0.2)
    env_y = np.maximum(env_y, 0.2)
    
    # Generate signals with independent phases
    x = env_x * np.sin(2 * np.pi * frequency * t + np.random.uniform(0, 2*np.pi)) + 0.1 * np.random.randn(n_samples)
    y = env_y * np.sin(2 * np.pi * frequency * t + np.random.uniform(0, 2*np.pi)) + 0.1 * np.random.randn(n_samples)
    
    return x, y

In [ ]:
# Visualization 5: Power correlation example

np.random.seed(42)
fs = 500.0
n_samples = 5000
band = (8, 12)

x, y = generate_power_correlated_signals(n_samples, fs, 10.0, 0.8, seed=42)
t = np.arange(n_samples) / fs

# Compute power
pow_x = compute_instantaneous_power(x, fs, band)
pow_y = compute_instantaneous_power(y, fs, band)
powcorr = stats.pearsonr(pow_x, pow_y)[0]

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Panel 1: Filtered signals
ax1 = axes[0, 0]
x_filt = bandpass_filter(x, fs, band)
y_filt = bandpass_filter(y, fs, band)
ax1.plot(t, x_filt, color=COLORS['signal_1'], linewidth=1, alpha=0.8, label='Signal X')
ax1.plot(t, y_filt, color=COLORS['signal_2'], linewidth=1, alpha=0.8, label='Signal Y')
ax1.set_xlabel('Time (s)', fontsize=11)
ax1.set_ylabel('Amplitude', fontsize=11)
ax1.set_title('Filtered Signals (8-12 Hz)', fontsize=12, fontweight='bold')
ax1.legend(loc='upper right')
ax1.set_xlim(0, 5)

# Panel 2: Power time series
ax2 = axes[0, 1]
ax2.plot(t, pow_x, color=COLORS['signal_1'], linewidth=1.5, label='Power X')
ax2.plot(t, pow_y, color=COLORS['signal_2'], linewidth=1.5, label='Power Y')
ax2.set_xlabel('Time (s)', fontsize=11)
ax2.set_ylabel('Power', fontsize=11)
ax2.set_title('Instantaneous Power P(t) = A(t)²', fontsize=12, fontweight='bold')
ax2.legend(loc='upper right')

# Panel 3: Power scatter
ax3 = axes[1, 0]
ax3.scatter(pow_x, pow_y, alpha=0.2, color=COLORS['accent'], s=5)
# Fit line
z = np.polyfit(pow_x, pow_y, 1)
p = np.poly1d(z)
x_line = np.linspace(pow_x.min(), pow_x.max(), 100)
ax3.plot(x_line, p(x_line), color=COLORS['highlight'], linewidth=2, label=f'r = {powcorr:.3f}')
ax3.set_xlabel('Power X', fontsize=11)
ax3.set_ylabel('Power Y', fontsize=11)
ax3.set_title('Power Correlation', fontsize=12, fontweight='bold')
ax3.legend(loc='upper left')

# Panel 4: Result
ax4 = axes[1, 1]
env_x = extract_envelope(x_filt)
env_y = extract_envelope(y_filt)
ccorr = stats.pearsonr(env_x, env_y)[0]

bars = ax4.bar(['CCorr\n(Envelope)', 'PowCorr\n(Power)'], [ccorr, powcorr],
               color=[COLORS['signal_1'], COLORS['accent']], edgecolor='white', linewidth=2)
for bar, val in zip(bars, [ccorr, powcorr]):
    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
             f'{val:.3f}', ha='center', fontsize=12, fontweight='bold')
ax4.set_ylabel('Correlation', fontsize=11)
ax4.set_title('Comparison: Envelope vs Power Correlation', fontsize=12, fontweight='bold')
ax4.set_ylim(0, 1.1)

plt.suptitle('Power Correlation Example', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Visualization 6: When CCorr and PowCorr diverge

np.random.seed(42)

# Scenario: Add low-amplitude noise that affects envelope more than power
n_samples = 10000
fs = 500.0
t = np.arange(n_samples) / fs
band = (8, 12)

# Clean signals with high coupling
shared_mod = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
x_clean = shared_mod * np.sin(2 * np.pi * 10 * t)
y_clean = shared_mod * np.sin(2 * np.pi * 10 * t + np.pi/4)

# Add different noise levels
noise_levels = [0.0, 0.3, 0.6, 1.0]
ccorrs = []
powcorrs = []

for noise in noise_levels:
    x = x_clean + noise * np.random.randn(n_samples)
    y = y_clean + noise * np.random.randn(n_samples)
    
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    
    env_x = extract_envelope(x_filt)
    env_y = extract_envelope(y_filt)
    pow_x = env_x ** 2
    pow_y = env_y ** 2
    
    ccorrs.append(stats.pearsonr(env_x, env_y)[0])
    powcorrs.append(stats.pearsonr(pow_x, pow_y)[0])

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Line plot
ax1 = axes[0]
ax1.plot(noise_levels, ccorrs, 'o-', color=COLORS['signal_1'], linewidth=2, markersize=10, label='CCorr (Envelope)')
ax1.plot(noise_levels, powcorrs, 's-', color=COLORS['accent'], linewidth=2, markersize=10, label='PowCorr (Power)')
ax1.set_xlabel('Noise Level (std)', fontsize=12)
ax1.set_ylabel('Correlation', fontsize=12)
ax1.set_title('Effect of Noise on Envelope vs Power Correlation', fontsize=12, fontweight='bold')
ax1.legend(loc='lower left', fontsize=11)
ax1.set_ylim(0, 1.05)
ax1.grid(True, alpha=0.3)

# Difference
ax2 = axes[1]
diff = np.array(powcorrs) - np.array(ccorrs)
colors_bar = [COLORS['highlight'] if d > 0 else COLORS['signal_2'] for d in diff]
bars = ax2.bar([f'{n}' for n in noise_levels], diff, color=colors_bar, edgecolor='white', linewidth=2)
ax2.axhline(0, color='black', linestyle='-', linewidth=1)
for bar, val in zip(bars, diff):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005 * np.sign(val),
             f'{val:.3f}', ha='center', fontsize=10, fontweight='bold')
ax2.set_xlabel('Noise Level', fontsize=12)
ax2.set_ylabel('PowCorr - CCorr', fontsize=12)
ax2.set_title('Difference: Power correlation often slightly higher', fontsize=12, fontweight='bold')

plt.suptitle('When Envelope and Power Correlation Diverge', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("💡 Power correlation tends to be slightly more robust to noise")
print("   because squaring reduces relative noise contribution.")

---

## Section 6: Volume Conduction and Orthogonalization

Just like envelope correlation, **power correlation is inflated by volume conduction**. The same solution applies: **orthogonalization**.

In [ ]:
def compute_orthogonalized_power_correlation(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    symmetric: bool = True
) -> float:
    """
    Compute power correlation with orthogonalization.
    
    Parameters
    ----------
    x : NDArray[np.float64]
        First signal
    y : NDArray[np.float64]
        Second signal
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    symmetric : bool, optional
        If True, average both directions (default: True)
        
    Returns
    -------
    float
        Orthogonalized power correlation in range [-1, 1]
    """
    # Bandpass filter
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    
    # Analytic signals
    z_x = signal.hilbert(x_filt)
    z_y = signal.hilbert(y_filt)
    
    # Phases and envelopes
    phase_x = np.angle(z_x)
    phase_y = np.angle(z_y)
    env_x = np.abs(z_x)
    
    # Orthogonalize y with respect to x
    y_orth = np.imag(z_y * np.exp(-1j * phase_x))
    env_y_orth = np.abs(y_orth)
    
    # Power
    pow_x = env_x ** 2
    pow_y_orth = env_y_orth ** 2
    
    # Correlation
    if np.std(pow_x) == 0 or np.std(pow_y_orth) == 0:
        corr1 = 0.0
    else:
        corr1 = stats.pearsonr(pow_x, pow_y_orth)[0]
    
    if symmetric:
        # Also compute in other direction
        env_y = np.abs(z_y)
        x_orth = np.imag(z_x * np.exp(-1j * phase_y))
        env_x_orth = np.abs(x_orth)
        
        pow_y = env_y ** 2
        pow_x_orth = env_x_orth ** 2
        
        if np.std(pow_y) == 0 or np.std(pow_x_orth) == 0:
            corr2 = 0.0
        else:
            corr2 = stats.pearsonr(pow_y, pow_x_orth)[0]
        
        return float((corr1 + corr2) / 2)
    
    return float(corr1)

In [ ]:
# Visualization 7: Volume conduction test

np.random.seed(42)
fs = 500.0
n_samples = 10000
t = np.arange(n_samples) / fs
band = (8, 12)

# Volume conduction scenario
source = (1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)) * np.sin(2 * np.pi * 10 * t)
x_vc = source + 0.1 * np.random.randn(n_samples)
y_vc = 0.9 * source + 0.1 * np.random.randn(n_samples)

# True connection scenario
mod = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
x_true = mod * np.sin(2 * np.pi * 10 * t) + 0.2 * np.random.randn(n_samples)
y_true = mod * np.sin(2 * np.pi * 10 * t + np.pi/4) + 0.2 * np.random.randn(n_samples)

# Compute correlations
powcorr_vc_std = compute_power_correlation(x_vc, y_vc, fs, band)
powcorr_vc_orth = compute_orthogonalized_power_correlation(x_vc, y_vc, fs, band)
powcorr_true_std = compute_power_correlation(x_true, y_true, fs, band)
powcorr_true_orth = compute_orthogonalized_power_correlation(x_true, y_true, fs, band)

fig, ax = plt.subplots(figsize=(10, 6))

x_pos = np.array([0, 1, 3, 4])
values = [powcorr_vc_std, powcorr_vc_orth, powcorr_true_std, powcorr_true_orth]
colors_bar = [COLORS['warning'], COLORS['signal_1'], COLORS['warning'], COLORS['signal_1']]

bars = ax.bar(x_pos, values, color=colors_bar, edgecolor='white', linewidth=2, width=0.8)

for bar, val in zip(bars, values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.03,
            f'{val:.3f}', ha='center', fontsize=11, fontweight='bold')

ax.set_xticks([0.5, 3.5])
ax.set_xticklabels(['Volume Conduction\n(Should be LOW)', 'True Connection\n(Should remain HIGH)'], fontsize=11)
ax.set_ylabel('Power Correlation', fontsize=12)
ax.set_title('Orthogonalized Power Correlation is Robust to Volume Conduction', fontsize=14, fontweight='bold')
ax.set_ylim(0, 1.1)

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=COLORS['warning'], label='Standard'),
                   Patch(facecolor=COLORS['signal_1'], label='Orthogonalized')]
ax.legend(handles=legend_elements, loc='upper right', fontsize=11)

plt.tight_layout()
plt.show()

print("Volume Conduction:")
print(f"  Standard: {powcorr_vc_std:.3f} → Orthogonalized: {powcorr_vc_orth:.3f}")
print(f"\nTrue Connection:")
print(f"  Standard: {powcorr_true_std:.3f} → Orthogonalized: {powcorr_true_orth:.3f}")

---

## Section 7: Power Correlation Matrix

In [ ]:
def compute_power_correlation_matrix(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    orthogonalize: bool = True
) -> NDArray[np.float64]:
    """
    Compute power correlation matrix for all channel pairs.
    
    Parameters
    ----------
    data : NDArray[np.float64]
        EEG data with shape (n_channels, n_samples)
    fs : float
        Sampling frequency in Hz
    band : tuple[float, float]
        Frequency band (low, high) in Hz
    orthogonalize : bool, optional
        Whether to use orthogonalization (default: True)
        
    Returns
    -------
    NDArray[np.float64]
        Power correlation matrix
    """
    n_channels = data.shape[0]
    
    # Filter and get analytic signals
    data_filt = bandpass_filter(data, fs, band)
    z_data = signal.hilbert(data_filt, axis=-1)
    phases = np.angle(z_data)
    envelopes = np.abs(z_data)
    powers = envelopes ** 2
    
    # Initialize matrix
    powcorr_matrix = np.eye(n_channels)
    
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            if orthogonalize:
                # Orthogonalize j w.r.t. i
                j_orth = np.imag(z_data[j] * np.exp(-1j * phases[i]))
                pow_j_orth = np.abs(j_orth) ** 2
                corr1 = stats.pearsonr(powers[i], pow_j_orth)[0] if np.std(pow_j_orth) > 0 else 0
                
                # Orthogonalize i w.r.t. j
                i_orth = np.imag(z_data[i] * np.exp(-1j * phases[j]))
                pow_i_orth = np.abs(i_orth) ** 2
                corr2 = stats.pearsonr(powers[j], pow_i_orth)[0] if np.std(pow_i_orth) > 0 else 0
                
                powcorr = (corr1 + corr2) / 2
            else:
                powcorr = stats.pearsonr(powers[i], powers[j])[0]
            
            powcorr_matrix[i, j] = powcorr
            powcorr_matrix[j, i] = powcorr
    
    return powcorr_matrix


def compute_envelope_correlation_matrix(
    data: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    orthogonalize: bool = True
) -> NDArray[np.float64]:
    """Compute envelope correlation matrix (from H01)."""
    n_channels = data.shape[0]
    data_filt = bandpass_filter(data, fs, band)
    z_data = signal.hilbert(data_filt, axis=-1)
    phases = np.angle(z_data)
    envelopes = np.abs(z_data)
    
    ccorr_matrix = np.eye(n_channels)
    
    for i in range(n_channels):
        for j in range(i + 1, n_channels):
            if orthogonalize:
                j_orth = np.abs(np.imag(z_data[j] * np.exp(-1j * phases[i])))
                i_orth = np.abs(np.imag(z_data[i] * np.exp(-1j * phases[j])))
                c1 = stats.pearsonr(envelopes[i], j_orth)[0] if np.std(j_orth) > 0 else 0
                c2 = stats.pearsonr(envelopes[j], i_orth)[0] if np.std(i_orth) > 0 else 0
                ccorr = (c1 + c2) / 2
            else:
                ccorr = stats.pearsonr(envelopes[i], envelopes[j])[0]
            
            ccorr_matrix[i, j] = ccorr
            ccorr_matrix[j, i] = ccorr
    
    return ccorr_matrix

In [ ]:
# Generate multi-channel data
np.random.seed(42)
n_channels = 8
n_samples = 5000
fs = 500.0
t = np.arange(n_samples) / fs

data = np.zeros((n_channels, n_samples))

# Cluster 1 (ch 0-2): shared modulation
shared_mod_1 = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
for i in range(3):
    data[i] = shared_mod_1 * np.sin(2 * np.pi * 10 * t + np.random.uniform(0, 2*np.pi)) + 0.2 * np.random.randn(n_samples)

# Cluster 2 (ch 3-5): different shared modulation
shared_mod_2 = 1 + 0.5 * np.sin(2 * np.pi * 0.4 * t + 1.5)
for i in range(3, 6):
    data[i] = shared_mod_2 * np.sin(2 * np.pi * 10 * t + np.random.uniform(0, 2*np.pi)) + 0.2 * np.random.randn(n_samples)

# Independent (ch 6-7)
for i in range(6, 8):
    indep_mod = 1 + 0.5 * np.sin(2 * np.pi * (0.3 + 0.1*i) * t + np.random.uniform(0, 2*np.pi))
    data[i] = indep_mod * np.sin(2 * np.pi * 10 * t + np.random.uniform(0, 2*np.pi)) + 0.3 * np.random.randn(n_samples)

# Compute matrices
ccorr_mat = compute_envelope_correlation_matrix(data, fs, (8, 12), orthogonalize=True)
powcorr_mat = compute_power_correlation_matrix(data, fs, (8, 12), orthogonalize=True)
diff_mat = powcorr_mat - ccorr_mat

In [ ]:
# Visualization 8: Three matrices comparison

fig, axes = plt.subplots(1, 3, figsize=(15, 4.5))

# CCorr
ax1 = axes[0]
im1 = ax1.imshow(ccorr_mat, cmap='RdBu_r', vmin=-1, vmax=1, aspect='equal')
ax1.set_title('Envelope Correlation (CCorr)', fontsize=12, fontweight='bold')
ax1.set_xlabel('Channel', fontsize=11)
ax1.set_ylabel('Channel', fontsize=11)
ax1.set_xticks(range(n_channels))
ax1.set_yticks(range(n_channels))
for pos in [2.5, 5.5]:
    ax1.axhline(pos, color='white', linewidth=1)
    ax1.axvline(pos, color='white', linewidth=1)
plt.colorbar(im1, ax=ax1, shrink=0.8)

# PowCorr
ax2 = axes[1]
im2 = ax2.imshow(powcorr_mat, cmap='RdBu_r', vmin=-1, vmax=1, aspect='equal')
ax2.set_title('Power Correlation (PowCorr)', fontsize=12, fontweight='bold')
ax2.set_xlabel('Channel', fontsize=11)
ax2.set_ylabel('Channel', fontsize=11)
ax2.set_xticks(range(n_channels))
ax2.set_yticks(range(n_channels))
for pos in [2.5, 5.5]:
    ax2.axhline(pos, color='white', linewidth=1)
    ax2.axvline(pos, color='white', linewidth=1)
plt.colorbar(im2, ax=ax2, shrink=0.8)

# Difference
ax3 = axes[2]
max_diff = np.max(np.abs(diff_mat))
im3 = ax3.imshow(diff_mat, cmap='RdBu_r', vmin=-max_diff, vmax=max_diff, aspect='equal')
ax3.set_title('Difference (PowCorr - CCorr)', fontsize=12, fontweight='bold')
ax3.set_xlabel('Channel', fontsize=11)
ax3.set_ylabel('Channel', fontsize=11)
ax3.set_xticks(range(n_channels))
ax3.set_yticks(range(n_channels))
for pos in [2.5, 5.5]:
    ax3.axhline(pos, color='black', linewidth=1, alpha=0.3)
    ax3.axvline(pos, color='black', linewidth=1, alpha=0.3)
plt.colorbar(im3, ax=ax3, shrink=0.8)

plt.suptitle('Comparing Envelope and Power Correlation Matrices (Orthogonalized)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print(f"Mean absolute difference: {np.mean(np.abs(diff_mat[np.triu_indices(n_channels, k=1)])):.4f}")
print(f"Max absolute difference: {np.max(np.abs(diff_mat[np.triu_indices(n_channels, k=1)])):.4f}")

---

## Section 8: Power Correlation for Hyperscanning

In [ ]:
def compute_power_correlation_hyperscanning(
    data_p1: NDArray[np.float64],
    data_p2: NDArray[np.float64],
    fs: float,
    band: tuple[float, float],
    orthogonalize_within: bool = True
) -> dict[str, NDArray[np.float64]]:
    """
    Compute power correlation matrices for hyperscanning.
    
    Parameters
    ----------
    data_p1 : NDArray[np.float64]
        Data from participant 1, shape (n_channels, n_samples)
    data_p2 : NDArray[np.float64]
        Data from participant 2
    fs : float
        Sampling frequency
    band : tuple[float, float]
        Frequency band
    orthogonalize_within : bool, optional
        Orthogonalize within-brain (default: True)
        
    Returns
    -------
    dict[str, NDArray[np.float64]]
        within_p1, within_p2, between, full matrices
    """
    n_ch1 = data_p1.shape[0]
    n_ch2 = data_p2.shape[0]
    
    # Filter and analytic
    data_p1_filt = bandpass_filter(data_p1, fs, band)
    data_p2_filt = bandpass_filter(data_p2, fs, band)
    
    z_p1 = signal.hilbert(data_p1_filt, axis=-1)
    z_p2 = signal.hilbert(data_p2_filt, axis=-1)
    
    phases_p1 = np.angle(z_p1)
    phases_p2 = np.angle(z_p2)
    powers_p1 = np.abs(z_p1) ** 2
    powers_p2 = np.abs(z_p2) ** 2
    
    # Within P1
    within_p1 = np.eye(n_ch1)
    for i in range(n_ch1):
        for j in range(i + 1, n_ch1):
            if orthogonalize_within:
                j_orth = np.abs(np.imag(z_p1[j] * np.exp(-1j * phases_p1[i]))) ** 2
                i_orth = np.abs(np.imag(z_p1[i] * np.exp(-1j * phases_p1[j]))) ** 2
                c1 = stats.pearsonr(powers_p1[i], j_orth)[0] if np.std(j_orth) > 0 else 0
                c2 = stats.pearsonr(powers_p1[j], i_orth)[0] if np.std(i_orth) > 0 else 0
                pc = (c1 + c2) / 2
            else:
                pc = stats.pearsonr(powers_p1[i], powers_p1[j])[0]
            within_p1[i, j] = pc
            within_p1[j, i] = pc
    
    # Within P2
    within_p2 = np.eye(n_ch2)
    for i in range(n_ch2):
        for j in range(i + 1, n_ch2):
            if orthogonalize_within:
                j_orth = np.abs(np.imag(z_p2[j] * np.exp(-1j * phases_p2[i]))) ** 2
                i_orth = np.abs(np.imag(z_p2[i] * np.exp(-1j * phases_p2[j]))) ** 2
                c1 = stats.pearsonr(powers_p2[i], j_orth)[0] if np.std(j_orth) > 0 else 0
                c2 = stats.pearsonr(powers_p2[j], i_orth)[0] if np.std(i_orth) > 0 else 0
                pc = (c1 + c2) / 2
            else:
                pc = stats.pearsonr(powers_p2[i], powers_p2[j])[0]
            within_p2[i, j] = pc
            within_p2[j, i] = pc
    
    # Between (no orthogonalization needed)
    between = np.zeros((n_ch1, n_ch2))
    for i in range(n_ch1):
        for j in range(n_ch2):
            between[i, j] = stats.pearsonr(powers_p1[i], powers_p2[j])[0]
    
    # Full matrix
    n_total = n_ch1 + n_ch2
    full = np.zeros((n_total, n_total))
    full[:n_ch1, :n_ch1] = within_p1
    full[n_ch1:, n_ch1:] = within_p2
    full[:n_ch1, n_ch1:] = between
    full[n_ch1:, :n_ch1] = between.T
    
    return {
        'within_p1': within_p1,
        'within_p2': within_p2,
        'between': between,
        'full': full
    }

In [ ]:
# Generate hyperscanning data
np.random.seed(42)
n_channels_per = 4
n_samples = 5000
fs = 500.0
t = np.arange(n_samples) / fs

# Shared between-brain modulation
shared_mod = 1 + 0.3 * np.sin(2 * np.pi * 0.2 * t)

# P1
p1_mod = 1 + 0.4 * np.sin(2 * np.pi * 0.3 * t)
data_p1 = np.zeros((n_channels_per, n_samples))
for i in range(n_channels_per):
    combined = 0.5 * p1_mod + 0.3 * shared_mod + 0.2
    data_p1[i] = combined * np.sin(2 * np.pi * 10 * t + np.random.uniform(0, 2*np.pi)) + 0.15 * np.random.randn(n_samples)

# P2
p2_mod = 1 + 0.4 * np.sin(2 * np.pi * 0.35 * t + 1.0)
data_p2 = np.zeros((n_channels_per, n_samples))
for i in range(n_channels_per):
    combined = 0.5 * p2_mod + 0.3 * shared_mod + 0.2
    data_p2[i] = combined * np.sin(2 * np.pi * 10 * t + np.random.uniform(0, 2*np.pi)) + 0.15 * np.random.randn(n_samples)

# Compute
hyper_powcorr = compute_power_correlation_hyperscanning(data_p1, data_p2, fs, (8, 12))

In [ ]:
# Visualization 9: Hyperscanning power correlation

fig, ax = plt.subplots(figsize=(8, 7))

im = ax.imshow(hyper_powcorr['full'], cmap='RdBu_r', vmin=-1, vmax=1, aspect='equal')

# Block separators
ax.axhline(n_channels_per - 0.5, color='black', linewidth=2)
ax.axvline(n_channels_per - 0.5, color='black', linewidth=2)

# Labels
n_total = 2 * n_channels_per
labels = [f'P1-{i}' for i in range(n_channels_per)] + [f'P2-{i}' for i in range(n_channels_per)]
ax.set_xticks(range(n_total))
ax.set_yticks(range(n_total))
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_yticklabels(labels)

# Block annotations
ax.text(1.5, 1.5, 'Within\nP1', ha='center', va='center', fontsize=9, fontweight='bold')
ax.text(5.5, 5.5, 'Within\nP2', ha='center', va='center', fontsize=9, fontweight='bold')
ax.text(5.5, 1.5, 'Between', ha='center', va='center', fontsize=9, fontweight='bold')

ax.set_title('Inter-Brain Power Correlation', fontsize=14, fontweight='bold')
plt.colorbar(im, ax=ax, shrink=0.8, label='PowCorr')

plt.tight_layout()
plt.show()

# Summary stats
mean_within_p1 = np.mean(hyper_powcorr['within_p1'][np.triu_indices(n_channels_per, k=1)])
mean_within_p2 = np.mean(hyper_powcorr['within_p2'][np.triu_indices(n_channels_per, k=1)])
mean_between = np.mean(hyper_powcorr['between'])

print(f"Mean power correlation:")
print(f"  Within P1: {mean_within_p1:.3f}")
print(f"  Within P2: {mean_within_p2:.3f}")
print(f"  Between:   {mean_between:.3f}")

---

## Section 9: Choosing Between Amplitude Metrics

### Envelope Correlation (CCorr)
- Most commonly used in literature
- Linear in amplitude
- Easier to compare with existing studies

### Power Correlation (PowCorr)
- Emphasizes larger fluctuations (squared)
- May be more robust to low-amplitude noise
- Matches spectral analysis (which uses power)

**Recommendation**: CCorr is the standard choice. PowCorr is an alternative when robustness to noise is important. Always report which metric you used!

---

## Section 10: Complete Connectivity Comparison

In [ ]:
def compute_all_amplitude_metrics(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> dict[str, float]:
    """
    Compute all amplitude connectivity metrics.
    
    Returns
    -------
    dict[str, float]
        ccorr, ccorr_orth, powcorr, powcorr_orth
    """
    # Filter and extract
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    
    env_x = extract_envelope(x_filt)
    env_y = extract_envelope(y_filt)
    pow_x = env_x ** 2
    pow_y = env_y ** 2
    
    ccorr = stats.pearsonr(env_x, env_y)[0]
    powcorr = stats.pearsonr(pow_x, pow_y)[0]
    
    # Orthogonalized (simplified)
    z_x = signal.hilbert(x_filt)
    z_y = signal.hilbert(y_filt)
    phase_x = np.angle(z_x)
    phase_y = np.angle(z_y)
    
    y_orth = np.abs(np.imag(z_y * np.exp(-1j * phase_x)))
    x_orth = np.abs(np.imag(z_x * np.exp(-1j * phase_y)))
    
    ccorr_orth = (stats.pearsonr(env_x, y_orth)[0] + stats.pearsonr(env_y, x_orth)[0]) / 2
    powcorr_orth = (stats.pearsonr(pow_x, y_orth**2)[0] + stats.pearsonr(pow_y, x_orth**2)[0]) / 2
    
    return {
        'ccorr': ccorr,
        'ccorr_orth': ccorr_orth,
        'powcorr': powcorr,
        'powcorr_orth': powcorr_orth
    }


def compute_all_connectivity_metrics(
    x: NDArray[np.float64],
    y: NDArray[np.float64],
    fs: float,
    band: tuple[float, float]
) -> dict[str, float]:
    """
    Compute complete connectivity profile (phase + amplitude).
    
    Returns
    -------
    dict[str, float]
        All phase and amplitude metrics
    """
    # Filter
    x_filt = bandpass_filter(x, fs, band)
    y_filt = bandpass_filter(y, fs, band)
    
    # Analytic signals
    z_x = signal.hilbert(x_filt)
    z_y = signal.hilbert(y_filt)
    
    phase_x = np.angle(z_x)
    phase_y = np.angle(z_y)
    env_x = np.abs(z_x)
    env_y = np.abs(z_y)
    
    phase_diff = phase_x - phase_y
    
    # Phase metrics
    plv = float(np.abs(np.mean(np.exp(1j * phase_diff))))
    
    signs = np.sign(np.sin(phase_diff))
    pli = float(np.abs(np.mean(signs)))
    
    sin_diff = np.sin(phase_diff)
    wpli = float(np.abs(np.sum(sin_diff)) / np.sum(np.abs(sin_diff))) if np.sum(np.abs(sin_diff)) > 0 else 0
    
    # Amplitude metrics
    pow_x = env_x ** 2
    pow_y = env_y ** 2
    
    ccorr = stats.pearsonr(env_x, env_y)[0]
    powcorr = stats.pearsonr(pow_x, pow_y)[0]
    
    # Orthogonalized
    y_orth = np.abs(np.imag(z_y * np.exp(-1j * phase_x)))
    x_orth = np.abs(np.imag(z_x * np.exp(-1j * phase_y)))
    
    ccorr_orth = (stats.pearsonr(env_x, y_orth)[0] + stats.pearsonr(env_y, x_orth)[0]) / 2
    powcorr_orth = (stats.pearsonr(pow_x, y_orth**2)[0] + stats.pearsonr(pow_y, x_orth**2)[0]) / 2
    
    return {
        'plv': plv,
        'pli': pli,
        'wpli': wpli,
        'ccorr': ccorr,
        'ccorr_orth': ccorr_orth,
        'powcorr': powcorr,
        'powcorr_orth': powcorr_orth
    }

In [ ]:
# Visualization 11: Complete connectivity profile

np.random.seed(42)
fs = 500.0
n_samples = 10000
t = np.arange(n_samples) / fs
band = (8, 12)

# Create signals with both phase and amplitude coupling
shared_mod = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
x = shared_mod * np.sin(2 * np.pi * 10 * t) + 0.2 * np.random.randn(n_samples)
y = shared_mod * np.sin(2 * np.pi * 10 * t + np.pi/6) + 0.2 * np.random.randn(n_samples)

# Compute all metrics
all_metrics = compute_all_connectivity_metrics(x, y, fs, band)

fig, ax = plt.subplots(figsize=(12, 6))

# Group metrics
phase_names = ['PLV', 'PLI', 'wPLI']
phase_values = [all_metrics['plv'], all_metrics['pli'], all_metrics['wpli']]

amp_names = ['CCorr', 'CCorr-orth', 'PowCorr', 'PowCorr-orth']
amp_values = [all_metrics['ccorr'], all_metrics['ccorr_orth'], 
              all_metrics['powcorr'], all_metrics['powcorr_orth']]

all_names = phase_names + [''] + amp_names  # Gap between groups
all_values = phase_values + [0] + amp_values
colors = [COLORS['signal_1']]*3 + ['white'] + [COLORS['accent']]*4

x_pos = np.arange(len(all_names))
bars = ax.bar(x_pos, all_values, color=colors, edgecolor=['black']*3 + ['white'] + ['black']*4, linewidth=2)

# Labels
for i, (bar, val) in enumerate(zip(bars, all_values)):
    if val > 0:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02,
                f'{val:.3f}', ha='center', fontsize=10, fontweight='bold')

ax.set_xticks(x_pos)
ax.set_xticklabels(all_names, fontsize=10)
ax.set_ylabel('Value', fontsize=12)
ax.set_title('Complete Connectivity Profile\n(Phase + Amplitude Metrics)', fontsize=14, fontweight='bold')
ax.set_ylim(0, 1.1)

# Group labels
ax.text(1, -0.15, 'PHASE', ha='center', fontsize=11, fontweight='bold', color=COLORS['signal_1'],
        transform=ax.get_xaxis_transform())
ax.text(5.5, -0.15, 'AMPLITUDE', ha='center', fontsize=11, fontweight='bold', color=COLORS['accent'],
        transform=ax.get_xaxis_transform())

plt.tight_layout()
plt.show()

print("\nComplete Connectivity Profile:")
print("="*40)
print("PHASE METRICS:")
for name in ['plv', 'pli', 'wpli']:
    print(f"  {name.upper()}: {all_metrics[name]:.3f}")
print("\nAMPLITUDE METRICS:")
for name in ['ccorr', 'ccorr_orth', 'powcorr', 'powcorr_orth']:
    print(f"  {name}: {all_metrics[name]:.3f}")

In [ ]:
# Visualization 12: Correlation matrix of metrics across many signal pairs

np.random.seed(42)
n_pairs = 100

metric_names = ['PLV', 'PLI', 'wPLI', 'CCorr', 'CCorr-orth', 'PowCorr', 'PowCorr-orth']
all_results = {name: [] for name in metric_names}

for _ in range(n_pairs):
    # Random coupling
    phase_coupling = np.random.uniform(0, 1)
    amp_coupling = np.random.uniform(0, 1)
    
    # Generate signals
    shared_mod = 1 + 0.5 * np.sin(2 * np.pi * 0.3 * t)
    indep_mod_x = 1 + 0.5 * np.sin(2 * np.pi * 0.4 * t + np.random.uniform(0, 2*np.pi))
    indep_mod_y = 1 + 0.5 * np.sin(2 * np.pi * 0.5 * t + np.random.uniform(0, 2*np.pi))
    
    env_x = np.sqrt(amp_coupling) * shared_mod + np.sqrt(1-amp_coupling) * indep_mod_x
    env_y = np.sqrt(amp_coupling) * shared_mod + np.sqrt(1-amp_coupling) * indep_mod_y
    
    if phase_coupling > 0.5:
        phase_lag = np.pi/4
    else:
        phase_lag = np.random.uniform(0, 2*np.pi)
    
    x = env_x * np.sin(2 * np.pi * 10 * t) + 0.2 * np.random.randn(n_samples)
    y = env_y * np.sin(2 * np.pi * 10 * t + phase_lag) + 0.2 * np.random.randn(n_samples)
    
    metrics = compute_all_connectivity_metrics(x, y, fs, band)
    
    all_results['PLV'].append(metrics['plv'])
    all_results['PLI'].append(metrics['pli'])
    all_results['wPLI'].append(metrics['wpli'])
    all_results['CCorr'].append(metrics['ccorr'])
    all_results['CCorr-orth'].append(metrics['ccorr_orth'])
    all_results['PowCorr'].append(metrics['powcorr'])
    all_results['PowCorr-orth'].append(metrics['powcorr_orth'])

# Compute correlation matrix
metric_corr = np.zeros((7, 7))
for i, name_i in enumerate(metric_names):
    for j, name_j in enumerate(metric_names):
        metric_corr[i, j] = np.corrcoef(all_results[name_i], all_results[name_j])[0, 1]

fig, ax = plt.subplots(figsize=(9, 8))

im = ax.imshow(metric_corr, cmap='RdBu_r', vmin=-1, vmax=1, aspect='equal')

# Add values
for i in range(7):
    for j in range(7):
        color = 'white' if abs(metric_corr[i, j]) > 0.5 else 'black'
        ax.text(j, i, f'{metric_corr[i, j]:.2f}', ha='center', va='center', 
                fontsize=9, color=color, fontweight='bold')

ax.set_xticks(range(7))
ax.set_yticks(range(7))
ax.set_xticklabels(metric_names, rotation=45, ha='right', fontsize=10)
ax.set_yticklabels(metric_names, fontsize=10)

# Group separators
ax.axhline(2.5, color='black', linewidth=2)
ax.axvline(2.5, color='black', linewidth=2)

ax.set_title('Relationships Between Connectivity Metrics\n'
             '(Phase metrics cluster; Amplitude metrics cluster)', fontsize=13, fontweight='bold')
plt.colorbar(im, ax=ax, shrink=0.8, label='Correlation')

plt.tight_layout()
plt.show()

print("\n💡 Key observations:")
print("   - Phase metrics (PLV, PLI, wPLI) correlate with each other")
print("   - Amplitude metrics (CCorr, PowCorr) correlate with each other")
print("   - Phase and amplitude show weaker cross-correlation")
print("   - They capture DIFFERENT aspects of connectivity!")

---

## Section 11: Hands-On Exercises

In [ ]:
# Exercise 1: Power Correlation Basics
# Generate signals with correlated power modulation
# Compute instantaneous power correlation

np.random.seed(42)

# YOUR CODE HERE

print("Exercise 1: Basic power correlation")

In [ ]:
# Exercise 2: Instantaneous vs Windowed
# Compare both methods on the same signals

np.random.seed(42)

# YOUR CODE HERE
# Discuss: When would you prefer windowed over instantaneous?

print("Exercise 2: Instantaneous vs windowed comparison")

In [ ]:
# Exercise 3: CCorr vs PowCorr
# Generate signals where the two metrics diverge
# Hint: add low-amplitude noise

np.random.seed(42)

# YOUR CODE HERE

print("Exercise 3: CCorr vs PowCorr divergence")

In [ ]:
# Exercise 4: Orthogonalization Check
# Simulate volume conduction
# Verify orthogonalized PowCorr is robust

np.random.seed(42)

# YOUR CODE HERE

print("Exercise 4: Volume conduction test")

In [ ]:
# Exercise 5: Complete Metric Comparison
# Create channel pairs with different connectivity types
# Compute ALL metrics and interpret

np.random.seed(42)

# YOUR CODE HERE

print("Exercise 5: Complete metric comparison")

In [ ]:
# Exercise 6: Hyperscanning Application
# Compare CCorr and PowCorr for hyperscanning

np.random.seed(42)

# YOUR CODE HERE
# Which metric would you report and why?

print("Exercise 6: Hyperscanning comparison")

---

## Summary

### Key Takeaways

1. **Power correlation**: Correlation of squared amplitude (power) time series
   - Power = A² emphasizes larger fluctuations
   - Range: -1 to +1

2. **Two approaches**:
   - Instantaneous: continuous P(t), high temporal resolution
   - Windowed: averaged power per epoch, more robust

3. **Highly related to envelope correlation** — usually similar values
   - PowCorr may be slightly more robust to low-amplitude noise

4. **Orthogonalization** addresses volume conduction (like CCorr)

5. **For hyperscanning**: Captures shared power fluctuations between participants

6. **Choice**: CCorr is the standard; PowCorr is an alternative
   - Always report which metric you used!

7. **Complete toolkit now**:
   - Phase: PLV, PLI, wPLI
   - Amplitude: CCorr, CCorr-orth, PowCorr, PowCorr-orth
   - Different metrics capture different aspects of connectivity!

---

## Discussion Questions

1. In what scenarios would you expect envelope correlation and power correlation to give notably different results? What signal characteristics would drive this difference?

2. You're designing a hyperscanning study and need to choose connectivity metrics. Would you compute both phase and amplitude metrics? How would you interpret results if they conflict?

3. The literature mostly reports envelope correlation (AEC). Is there value in also reporting power correlation, or does it add unnecessary complexity?

4. Power is A². What about using log-power (dB) for correlation? Would this be more or less sensitive to large fluctuations? When might log-power correlation be useful?

5. Looking across all the metrics covered in this workshop (PLV, PLI, wPLI, CCorr, PowCorr), which would you recommend as a "default" set for a new hyperscanning study? Justify your choices.

---

## 🎉 Workshop Complete!

Congratulations! You've completed the hyperscanning connectivity metrics workshop. You now have a comprehensive toolkit for analyzing neural connectivity:

**Phase-based metrics** (timing alignment):
- PLV: Simple, high sensitivity
- PLI: Volume conduction robust
- wPLI: Robust to both VC and noise

**Amplitude-based metrics** (strength co-modulation):
- CCorr: Standard amplitude coupling
- PowCorr: Power-based alternative
- Orthogonalized versions for VC robustness

**Remember**: These metrics capture *different* aspects of neural connectivity. Use them together for a complete picture!